In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar

# from gcsa.recurrence import Recurrence, DAILY, SU, SA
from dotenv import load_dotenv
import os

load_dotenv()

personal_cal = GoogleCalendar(os.getenv("PERSONAL_EMAIL"))


In [ ]:
from datetime import timedelta, date

events = personal_cal.get_events(
    date.today(), date.today() + timedelta(weeks=5), order_by="updated"
)

calendly_events = []
for event in events:
    if event.description is not None and "Calendly" in event.description:
        calendly_events.append(event)


In [ ]:
for event in calendly_events:
    has_attendee = False
    for attendee in event.attendees:
        if attendee.email == "eric.ma@modernatx.com":
            has_attendee = True
            break
    if not has_attendee:
        event.add_attendee("eric.ma@modernatx.com")
        personal_cal.update_event(event, send_updates="all")


In [ ]:
event = calendly_events[0]

attendee = event.attendees[0]
wanted_attendees = [a for a in event.attendees if a.email != "eric.ma@modernatx.com"]
event.attendees = wanted_attendees
personal_cal.update_event(event, send_updates="all")


In [ ]:
def ensure_work_email_on_calendly_events():
    personal_cal = GoogleCalendar(calendar_id=os.getenv("PERSONAL_EMAIL"))
    calendly_events = personal_cal.get_events_within(
        timedelta(weeks=5),
        filter_funcs=[
            lambda event: "description" in event and "Calendly" in event["description"]
        ],
    )
    for event in calendly_events:
        event.invite(os.getenv("WORK_EMAIL"))


ensure_work_email_on_calendly_events()


In [ ]:
events = personal_cal.get_events_within(
    timedelta(weeks=5),
    filter_funcs=[
        lambda event: "description" in event
        and "reclaim.ai" not in event["description"],
        lambda event: "summary" in event
        and event["summary"] not in ["BIKE TO WORK", "Weekly Vision"],
    ],
)


In [ ]:
from markdown import markdown


def sync_altwork_with_personal_and_work():
    """
    For every meeting in altwork, ensure that there is a corresponding entry
    in personal and work calendars but with a codename for it.
    """
    # Instantiate APIs for personal and altwork calendars.
    personal_cal = GoogleCalendar(calendar_id=os.getenv("PERSONAL_EMAIL"))
    altwork_cal = GoogleCalendar(calendar_id=os.getenv("ALTWORK_EMAIL"))

    # Find all Takeda events from the altwork calendar.
    events = altwork_cal.get_events_within(
        timedelta(weeks=16),
        filter_funcs=[
            lambda event: "summary" in event and "Takeda" in event["summary"]
        ],
    )

    # Clear out all original BLOCK LABS events from the personal calendar.

    # Firstly, get all of the BLOCK LABS events in the coming 16 weeks
    block_labs_events = personal_cal.get_events_within(
        timedelta(weeks=16),
        filter_funcs=[
            lambda event: "summary" in event and "BLOCK LABS" in event["summary"]
        ],
    )
    # Delete them all.
    for event in block_labs_events:
        event.delete()

    # Then, re-create all of the events as needed.
    for event in events:
        event = personal_cal.create_event(
            summary="BLOCK LABS",
            start_time=event["start"],
            end_time=event["end"],
            description=markdown(f"Original [link]({event['htmlLink']})"),
        )
        # Ensure my Moderna email address is included.
        event.invite(os.getenv("WORK_EMAIL"))


sync_altwork_with_personal_and_work()


In [ ]:
from tqdm.auto import tqdm


# Delete all events by Reclaim
def delete_reclaim_events(calendar: GoogleCalendar):
    """Delete all events by Reclaim."""
    reclaim_events = calendar.get_events_within(
        timedelta(weeks=4),
        filter_funcs=[
            lambda event: "description" in event
            and "app.reclaim.ai" in event["description"]
        ],
    )
    for event in tqdm(reclaim_events):
        event.delete()


delete_reclaim_events(personal_cal)


In [ ]:
# def create_post_meeting_debriefs():
#     """Create post-meeting debriefs for every Calendly event."""
calendly_events = personal_cal.get_events_within(
    time_delta=timedelta(weeks=12),
    filter_funcs=[
        lambda event: "description" in event and "Calendly" in event["description"]
    ],
)


In [ ]:
event = calendly_events[1]
# Create a new event that lasts for 15 minutes after the previous event.
personal_cal.create_event(
    summary=f"{event['summary']} debrief",
    start_time=event["end"],
)

# event.event["extendedProperties"] = {"private": {"meetingId": ""}}
# event.update()


In [ ]:
event
